__качаем объявления о продаже квартир с авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
place = 'sevastopol'
avito_path = place+'/kvartiry/prodam'

raw_data_path = 'data/raw/flat/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-11-28 13:10:12 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-11-28 13:10:12 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-11-28 13:10:14 | AdsListParser: downloader init
[INFO    ] 2022-11-28 13:10:14 | AvitoParser: init
[INFO    ] 2022-11-28 13:10:21 | AvitoParser: 82 pages for read
[INFO    ] 2022-11-28 13:10:21 | AdsListParser: start read and parse pages...
[INFO    ] 2022-11-28 13:10:23 | AdsListParser: read page 1
[INFO    ] 2022-11-28 13:10:26 | AdsListParser: read page 2
[INFO    ] 2022-11-28 13:10:28 | AdsListParser: read page 3
[INFO    ] 2022-11-28 13:10:31 | AdsListParser: read page 4
[INFO    ] 2022-11-28 13:10:34 | AdsListParser: read page 5
[INFO    ] 2022-11-28 13:10:37 | AdsListParser: read page 6
[INFO    ] 2022-11-28 13:10:39 | AdsListParser: read page 7
[INFO    ] 2022-11-28 13:10:42 | AdsListParser: read page 8
[INFO    ] 2022-11-28 13:10:44 | AdsListParser: read page 9
[INFO    ] 2022-11-28 13:10:47 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

3943
3596


In [8]:
df['place']=place

In [9]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [10]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts,place
118,2569582595,"Объявление «3-к. квартира, 61,3 м², 9/9 эт.» 21 фотография",6500000,,"пр-т Октябрьской Революции, 52А","Срочно! \n\nПродается просторная благоустроенная трёхкомнатная квартира в Гагаринском районе г. Севастополя на пр-те Октябрьской революции. \n\nСостояние очень хорошее, жилое. Квартира видовая, солнечная. Все комнаты раздельные. Две просторные застекленные лод",3,2022-11-28 13:20:28.619982,sevastopol
1451,2538905346,"Объявление «1-к. квартира, 18 м², 1/3 эт.» 12 фотографий",2450000,,"ул. Военных Строителей, 3Б/4",Продается студия 18 м² Гагаринский район. Ул. Военных строителей 3Б/5.\nСвет вода газ тепло интернет Есть. На всё свои счётчики. Дом сдан! Ижс.\nОбщая площадь 18 м² Высота потолков 3м. Море школы детские сады магазины Рядом. Можно сразу прописаться. Соседи у,32,2022-11-28 13:20:28.619982,sevastopol


In [11]:
# !pip install dash_table